Looking at the state of the Kaggle forums of late, with [an explosion of AI Generated Text](https://www.kaggle.com/discussions/general/398579), I think the time has come to replace myself with the [Gemma LLM](https://storage.googleapis.com/deepmind-media/gemma/gemma-report.pdf) and from now on dedicate my time to more worthwhile endeavors.
The quality of my replies will now be even worse than usual, and I will receive fewer medals, but I was never in it for the medals in the first place.

Here we shall fine tune the [Gemma](https://www.kaggle.com/models/google/gemma) model with my historical replies scraped from the [Meta Kaggle](https://www.kaggle.com/datasets/kaggle/meta-kaggle) dataset.
This work is extremely heavily based of the following two magnificent notebooks by [Nilay Chauhan](https://www.kaggle.com/nilaychauhan)
* [Get started with Gemma using KerasNLP](https://www.kaggle.com/code/nilaychauhan/get-started-with-gemma-using-kerasnlp)
* [Fine-tune Gemma models in Keras using LoRA](https://www.kaggle.com/code/nilaychauhan/fine-tune-gemma-models-in-keras-using-lora)

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 256)
import re
# from https://www.kaggle.com/code/nilaychauhan/fine-tune-gemma-models-in-keras-using-lora

# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

import keras
import keras_nlp

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.
tensorflowjs 4.16.0 requires packaging~=23.1, but you have packaging 21.3 which is incompatible.


2024-02-23 07:28:10.510246: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 07:28:10.510355: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 07:28:10.634999: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
%%time

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


CPU times: user 6.81 s, sys: 9.47 s, total: 16.3 s
Wall time: 59.6 s


### Before...

In [3]:
print(gemma_lm.generate("Will there be a private leaderboard shakeup?", max_length=256))

Will there be a private leaderboard shakeup?

I'm not sure if this is a bug or not, but I've noticed that the private leaderboard is not showing up in the game. I've tried restarting the game, but it still doesn't show up.

I've also noticed that the private leaderboard is not showing up in the game. I've tried restarting the game, but it still doesn't show up.

I've also noticed that the private leaderboard is not showing up in the game. I've tried restarting the game, but it still doesn't show up.

I've also noticed that the private leaderboard is not showing up in the game. I've tried restarting the game, but it still doesn't show up.

I've also noticed that the private leaderboard is not showing up in the game. I've tried restarting the game, but it still doesn't show up.

I've also noticed that the private leaderboard is not showing up in the game. I've tried restarting the game, but it still doesn't show up.

I've also noticed that the private leaderboard is not showing up in the

### Create a dataset of my Kaggle conversations obtained from [Meta Kaggle](https://www.kaggle.com/datasets/kaggle/meta-kaggle)

In [4]:
# Get my Kaggle User Id
Users = pd.read_csv("/kaggle/input/meta-kaggle/Users.csv")
User_Id = Users.query('UserName == "carlmcbrideellis"')["Id"].item()

In [5]:
ForumMessages = pd.read_csv("/kaggle/input/meta-kaggle/ForumMessages.csv")
CME_posts = ForumMessages[(ForumMessages['PostUserId'] == User_Id)] #["Message"]
# Select only my posts that were replies
CME_Response = CME_posts[CME_posts['ReplyToForumMessageId'].notna()]
CME_Response = CME_Response.rename(columns={'Message': 'Response'})
# and now get the correponding "prompt"
CME_Response = CME_Response.merge(ForumMessages, left_on='ReplyToForumMessageId', right_on='Id')
CME_Response = CME_Response.rename(columns={'Message': 'Instruction'})
data = CME_Response[["Instruction","Response"]].copy()

how many conversations are there?

In [6]:
data.shape[0]

1951

OK, almost 2 thousand

In [7]:
# do some basic cleaning

# remove any HTML/Markdown tags
data["Instruction"] = data["Instruction"].str.replace(r'<[^<>]*>', '', regex=True)
# remove any newline
data["Instruction"] = data["Instruction"].str.replace(r'\n',' ', regex=True)
# remove any @user tags
data["Instruction"] = data["Instruction"].str.replace(r'(?<=\s)@[\w]+|(?<=^)@[\w]+', '', regex=True)

# repeat same cleaning for the Response column as well
data["Response"] = data["Response"].str.replace(r'<[^<>]*>', '', regex=True)
data["Response"] = data["Response"].str.replace(r'\n',' ', regex=True)
data["Response"] = data["Response"].str.replace(r'(?<=\s)@[\w]+|(?<=^)@[\w]+', '', regex=True)

In [8]:
# take a look
data.tail(10)

,Instruction,Response
1941,"Irrespective of the ML model, finding it really difficult to beat the last value benchmark with gaps while doing the cross-validation. However, the model performance is better on Jan 23 LB as compared to the last value for Jan 23. Really confusing to ...","Dear Indeed a simple ""last value + population correction"" model looks very good as far as the Public LB is concerned. I am not at all going down that route, but I have a horrible feeling that I am going to get my backside handed to me… All the best a..."
1942,It's getting close ;) https://github.com/Kaggle/docker-python/pull/1231,"Dear 3.10 ¡excellent! (Marking as Solved) All the best, carl"
1943,This is also known an Classifier Two Sample Test https://arxiv.org/abs/1610.06545,"Dear Interesting paper; many thanks for the link! Un saludo muy cordial, carl"
1944,"Comment Hi everyone. Ever since I joined Kaggle, I’ve dealt with errors in the notebooks I uploaded to Kaggle after developing them in the Anaconda environment installed in my laptop. Almost all these errors are caused by having a different python vers...","Dear Another option is perhaps to use the same Docker container as that used by kaggle. Un saludo muy cordial, carl"
1945,"Appreciate the kind words Carl, and couldn't agree more on how awful that feeling is. Hopefully you have a way forward to salvage your competition! For a laugh, your @ mention and choice of thread title generated its own fleeting moment of panic for me...","Dear ""Hopefully you have a way forward to salvage your competition!"" Indeed; I managed to turn it into a learning experience on the dangers of using seemingly predictive columns in a model (phew) BTW: Did not at all intend to induce panic, but I ..."
1946,"We are excited to announce that we are experimenting with a new way to show you content in your newsfeed. Currently, we use a complex algorithm to determine which posts you see. This algorithm takes into account a variety of factors, such as the type o...","Dear ""Don’t worry I didn’t make everyone automatically follow me!"" I am going to follow you anyway; nice work! I agree with and people could end up in a filter bubble, or some sort of Matthew effect could take place where people with many follo..."
1947,"Hi, I don't know if anyone had a chance to notice but in recent days we have exceeded the limit of 500 grandmaster titles. Currently, 284 titles have been assigned in competitions, 35 in datasets, 118 in notebooks and 63 in discussions, which makes a r...","Dear According to my script that runs daily(*) there are a total of 422 kaggle Grandmasters if one accounts for multiple GM rank achievements . All the best, carl (*) New GM's may take a few days to appear due to dependencies on various log files."
1948,"Thank you for sharing the helpful script. I think your script is productive to providing dataset specialized in machine learning, but I wonder what randomly shuffling the dataset has a certain threshold. A certain threshold is that random_state 42. I a...","Dear The reason for randomly shuffling the dataset is to reduce the possibility of data leakage, for example if the dataset has a sorted column. As for 42… All the best, carl"
1949,"Hi , Have you tried using VPN ? I'm currently in an accessible region(Morocco🇲🇦), I used VPN to connect to a server in a restricted country (Sweden) and then I tried the Get an API key I encountered the same issue as you: Access restricted ...","What is the Kaggle/Google standpoint on using a VPN? Is this something we could do (In particular I am thinking of the context of generating training data for the LLM - Detect AI Generated Text competition, which would be shared with the other compe..."
1950,"Hi! I am going to this page: https://ai.google.dev/tutorials/setup and when I press ""Get an API key"" I am directed to a page that looks like in the attachment. FYI: I am logged in with my google account. Could you please advice! Many thanks!","Dear +1 🇪🇸 Spain 🇪🇸: Access

In [9]:
CME_dataset = []
    
for index, row in data.iterrows():
    instruction, response = row['Instruction'], row['Response']
    template = (f"Instruction:\n{instruction}\n\nResponse:\n{response}")
    CME_dataset.append(template)

### LoRA fine-tuning

In [10]:
# Enable LoRA for the model and set the LoRA rank to 64.
gemma_lm.backbone.enable_lora(rank=64)

In [11]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [12]:
%%time

gemma_lm.fit(CME_dataset, epochs=1, batch_size=1)

1951/1951 ━━━━━━━━━━━━━━━━━━━━ 1453s 734ms/step - loss: 1.0314 - sparse_categorical_accuracy: 0.4377
CPU times: user 24min 20s, sys: 3.23 s, total: 24min 23s
Wall time: 24min 15s


### ...and now...

In [13]:
print(gemma_lm.generate("Will there be a private leaderboard shakeup?", max_length=256))

Will there be a private leaderboard shakeup?

This is the first time I've seen this.

I'm not a developer, I'm not sure how the private leaderboard works (if it does) and I'm just curious.

I think I've found a bug.

The problem is the "private score" field. When I look at my score I have a 99.7% and the private score is a 99.5%.

When I click the "private" link I get an error:

"The private leaderboard does not support this action. Please check the leaderboard rules"

I'm not sure what this means but I think this may explain the private leaderboard score being off.

I've never seen a private leaderboard that works like this, it seems like a bug.


meh, I think that should do the trick...

Now, back to that tabular dataset notebook I was working on!

### Related reading
* [Gemma: Open Models Based on Gemini Research and Technology](https://storage.googleapis.com/deepmind-media/gemma/gemma-report.pdf)
* [RAG using Llama 2, Langchain and ChromaDB](https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb) by [Gabriel Preda](https://www.kaggle.com/gpreda)
* [Gabriel Preda "*Developing Kaggle Notebooks*", Packt Publishing Limited (2023)](https://www.packtpub.com/product/developing-kaggle-notebooks/9781805128519) (Chapter 10)
* [Building A Transformer (GPT) From Scratch](https://www.kaggle.com/code/kevinbnisch/building-a-transformer-gpt-from-scratch) by [TheItCrOw](https://www.kaggle.com/kevinbnisch)
* [Get started with Gemma using KerasNLP](https://www.kaggle.com/code/nilaychauhan/get-started-with-gemma-using-kerasnlp) by [Nilay Chauhan](https://www.kaggle.com/nilaychauhan)
* [Fine-tune Gemma models in Keras using LoRA](https://www.kaggle.com/code/nilaychauhan/fine-tune-gemma-models-in-keras-using-lora) by [Nilay Chauhan](https://www.kaggle.com/nilaychauhan)